In [1]:
import pandas as pd
import numpy as np
import os
import time
from tqdm import tqdm

In [2]:
column_names = ['ms','ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','thigh_horiz_fw','thigh_vert', 'thigh_horiz_lat','trunk_horiz_fw','trunk_vert', 'trunk_horiz_lat','annotation']
datadir = 'dataset_fog_release/dataset'
outputdir = 'sliding_window_dataset/'

subject_files = {'s6':['S06R01.txt','S06R02.txt'],'s7':['S07R01.txt','S07R02.txt'],
                 's8': ['S08R01.txt'],'s9': ['S09R01.txt'],'s10':['S10R01.txt']}
subj_file = subject_files["s6"][0]
filename = f'{datadir}/{subj_file}'
df = pd.read_csv(filename,sep=' ', names = column_names)

In [3]:
def extract_experiment_windows(annotations):
    
    start = None
    stop = None
    windows = []
    for i in range(len(annotations)):
        if annotations[i] != 0 and not start:
            start = i
        if annotations[i] == 0 and start:
            end = i - 1
            windows.append((start,end))
            start = None
            end = None
    return windows

In [4]:
def create_and_save_sliding_windows(subj_data_frame,filename, outputdir, window_size = 1000, columns=['ankle_vert','annotation']):
    
    experiment_intervals = extract_experiment_windows(subj_data_frame['annotation'])
    df_as_np = subj_data_frame.to_numpy()

    experiment_intervals = extract_experiment_windows(df['annotation'])
    numpy_df = df[columns].to_numpy()
    numpy_df_intervals = []
    X = []
    y = []
    start = time.time()
    for interval in experiment_intervals:
        numpy_df_intervals.append(numpy_df[interval[0]:interval[1]])
    for experiment,n in zip(numpy_df_intervals, range(len(numpy_df_intervals))):
        for i in tqdm(range(len(experiment) - window_size)):
            row = [a for a in experiment[i:i+window_size]]
            X.append(row)
            label = experiment[i+window_size][3]
            y.append(0 if label == 1 else 1)
        np.save(f'{outputdir}{filename[0:6]}experiment{n}_X.npy',X)
        np.save(f'{outputdir}{filename[0:6]}experiment{n}_y.npy',y)

    end = time.time()
    print(end-start)
create_and_save_sliding_windows(df, subj_file, outputdir ,columns = ['ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','annotation'])

100%|██████████████████████████████████████████████████████████████████████████| 50200/50200 [00:47<00:00, 1063.97it/s]


OSError: 200800000 requested and 0 written

In [ ]:
X = np.load('sliding_window_dataset/S01R01experiment0_X.npy')
y = np.load('sliding_window_dataset/S01R01experiment0_y.npy')

In [ ]:
print(X.shape)
print(y.shape)
print(X[0])
print(np.where(y==1)[0])



In [ ]:
column_names = ['ms','ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','thigh_horiz_fw','thigh_vert', 'thigh_horiz_lat','trunk_horiz_fw','trunk_vert', 'trunk_horiz_lat','annotation']
datadir = 'dataset_fog_release/dataset'
outputdir = 'sliding_window_dataset/'

subject_files = {'s10':['S10R01.txt']}
for subj in subject_files.keys():
    for fname in subject_files[subj]:
        path_to_file = f'{datadir}/{fname}'
        df = pd.read_csv(path_to_file,sep=' ', names = column_names)
        create_and_save_sliding_windows(df, fname, outputdir ,columns = ['ankle_horiz_fw','ankle_vert', 'ankle_horiz_lat','annotation'])